# 导入相关库

In [ ]:
import paddle
import numpy as np
import matplotlib.pyplot as plt

paddle.__version__

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


'2.0.0'

# ② 数据准备

## 2.1 数据加载和预处理

In [ ]:
import paddle.vision.transforms as T

# 数据的加载和预处理
transform = T.Compose([T.Resize((256, 256)), 
                        T.CenterCrop(224), 
                        T.Transpose(),
                        T.Normalize(mean=255, data_format='CHW', to_rgb=True),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

# 训练数据集
train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform)

# 评估数据集
eval_dataset = paddle.vision.datasets.MNIST(mode='test', transform=transform)

print('训练集样本量: {}，验证集样本量: {}'.format(len(train_dataset), len(eval_dataset)))

训练集样本量: 60000，验证集样本量: 10000


# ③ 模型选择和开发

## 3.1 模型组网

In [ ]:
network = paddle.vision.models.resnet18(num_classes=100,pretrained=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1263: UserWarning: Skip loading for fc.weight. fc.weight receives a shape [512, 1000], but the expected shape is [512, 100].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1263: UserWarning: Skip loading for fc.bias. fc.bias receives a shape [1000], but the expected shape is [100].
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))


## 模型网络结构可视化

In [ ]:
# 模型封装
model = paddle.Model(network)

# 模型可视化
#model.summary((1, 28, 28))

In [ ]:
# 配置优化器、损失函数、评估指标
model.prepare(paddle.optimizer.Adam(learning_rate=0.001, parameters=network.parameters()),
              paddle.nn.CrossEntropyLoss(),
              paddle.metric.Accuracy())
              
# 启动模型全流程训练
model.fit(train_dataset,  # 训练数据集
          eval_dataset,   # 评估数据集
          epochs=5,       # 训练的总轮次
          batch_size=64,  # 训练使用的批大小
          verbose=1)      # 日志展示形式

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/5


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:636: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step 938/938 [==============================] - loss: 0.0274 - acc: 0.9789 - 124ms/step         
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 157/157 [==============================] - loss: 0.0087 - acc: 0.9695 - 119ms/step         
Eval samples: 10000
Epoch 2/5
step 938/938 [==============================] - loss: 0.0403 - acc: 0.9895 - 122ms/step        
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 157/157 [==============================] - loss: 8.8554e-04 - acc: 0.9880 - 116ms/step       
Eval samples: 10000
Epoch 3/5
step 938/938 [==============================] - loss: 0.0059 - acc: 0.9907 - 122ms/step        
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 157/157 [==============================] - loss: 0.0013 - acc: 0.9919 - 120ms/

# ⑤ 模型评估测试

## 5.1 模型评估

In [ ]:
# 模型评估，根据prepare接口配置的loss和metric进行返回
result = model.evaluate(eval_dataset, verbose=1)

print(result)

Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 10000/10000 [==============================] - loss: 1.1921e-07 - acc: 0.9926 - 10ms/step         
Eval samples: 10000
{'loss': [1.1920928e-07], 'acc': 0.9926}


# ⑥ 部署上线

## 6.1 保存模型

In [ ]:
# 保存用于后续继续调优训练的模型
model.save('finetuning/mnist')

## 6.2 继续调优训练

In [ ]:
from paddle.static import InputSpec


# 模型封装，为了后面保存预测模型，这里传入了inputs参数
model_2 = paddle.Model(network, inputs=[InputSpec(shape=[-1, 28, 28], dtype='float32', name='image')])

# 加载之前保存的阶段训练模型
model_2.load('finetuning/mnist')

# 模型配置
model_2.prepare(paddle.optimizer.Adam(learning_rate=0.001, parameters=network.parameters()),
                paddle.nn.CrossEntropyLoss(),
                paddle.metric.Accuracy())

# 模型全流程训练
model_2.fit(train_dataset, 
            eval_dataset,
            epochs=2,
            batch_size=64,
            verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/2
step 938/938 [==============================] - loss: 0.0307 - acc: 0.9921 - 124ms/step         
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 157/157 [==============================] - loss: 6.8058e-05 - acc: 0.9936 - 120ms/step       
Eval samples: 10000
Epoch 2/2
step 938/938 [==============================] - loss: 0.0024 - acc: 0.9956 - 122ms/step        
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 157/157 [==============================] - loss: 2.0093e-05 - acc: 0.9933 - 119ms/step         
Eval samples: 10000


## 6.3 保存预测模型

In [ ]:
# 保存用于后续推理部署的模型
model_2.save('infer/mnist')

TypeError: can't pickle paddle.fluid.core_avx.BlockDesc objects

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=be243217-dd5d-47ab-9ef2-99bb2e76d181' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>